In [10]:
import numpy as np
import matplotlib.pyplot as plt
import datetime

%matplotlib inline

### History

* 2016.12.11: updated RFI clipper to new statistical clipping methods. seems to have a memory leak/running slower than realtime. observations during this time were not successful.
* 2017.01.24: updated RFI clipper to new method which learns the bandpass, and writes out learned bandpass.

### Logging

* Are there logs/diagnostics proving ALFABURST was actually observing when the logs say it was?
    * cognizeALFA.log is the UPPERLIMIT on the total observing time, this does not account for nodes being down. But, it seems as if abc0, abc1, and abc3 have been consistently up. abc2 has been down most of the time during observation.
* Are there intermediate diagnostic outputs? e.g. integrated spectra before the single pulse search?
    * no, as of Jan 24 2017 we are saving the RFI bandpass logs

### Observing

* How long have we observed with ALFABURST?
    * based on parseCognizeALFALog.py (from 2015-08-17 to 2017-01-24): 44 days, 8:04:00 (3830640.00 seconds)
    * What about the relibaility of the abcX nodes?
        * abc2 is the most flaky
        * the lower limit is that abc0 was the only node up, which givs us 2 beams
        * the upper limit, which we know we are nowhere near is 7 beams
        * assuming each beam detects the same number of events, using abc1 (which seems stable) we can make an arguement for the uptime of the different nodes absed on the hard drive usage of recorded events
        * as of Jan 24 2017, disk usage is:
            * abc0: 82% - 2.0
            * abc1: 62% - 2.0
            * abc2: 16% - 0.50
            * abc3: 16% (only 1 beam) - 0.50
            * using these ratios we have observed for 221 days if only considering a single beam

* What does the survey sky coverage look like? How much time has been out of the galactic plan?
    * `./plotSuveryCoverage.py ../data/cognize.pkl --gal_cut=10 --max=200`
        * Total observation time: 3830640 seconds
        * Galaxy Cut Observation time: 2097480 seconds

In [11]:
obsTime = 3830640. #seconds
outOfPlaneObsTime = 2097480. #seconds

print 'Total Obs Time:', str(datetime.timedelta(seconds=obsTime))
print 'Out of Plane Obs Time:', str(datetime.timedelta(seconds=outOfPlaneObsTime))
print 'Precent out of plane:', (outOfPlaneObsTime / obsTime) * 100.

Total Obs Time: 44 days, 8:04:00
Out of Plane Obs Time: 24 days, 6:38:00
Precent out of plane: 54.755341144


* How much sky have been covered based on the observation time and beam size (for a single pixel, and for the total ALFA beam)?

A single pixel is approximately 3.5' wide at FWHM.

In [16]:
pixelFWHMarea = np.pi * (3.5 / (60. * 2.))**2. # pi * r^2, r = (FWHM/2) in degrees
alfaFWHMarea = 7. * pixelFWHMarea

print 'FWHM (single beam, sq. degrees):', pixelFWHMarea
print 'FWHM (ALFA, sq. degrees):', alfaFWHMarea

FWHM (single beam, sq. degrees): 0.00267253541712
FWHM (ALFA, sq. degrees): 0.0187077479198


`./plotSuveryCoverage.py ../data/cognize.pkl --gal_cut=10 --max=200 --sampling --NSIDE=256`

Gridding onto a HEALPIX map (NSIDE=256) results in 11401 unique pointings. From (https://lambda.gsfc.nasa.gov/toolbox/tb_pixelcoords.cfm) NSIDE=256 results in 786432 pixels, mean spacing of 0.2290 degrees, which is an area of 1.5978967 x 10^-5 steradian. This is approximately the size of the full ALFA FWHM.

### Sensitivity

* What is the ALFA system temperature? Observing frequency? What is the sensitivity?
    * ALFABURST is 56 MHz centred at 1375 MHz[?] (Chennamangalam 2017)
    * 'ALFA is a seven-beam system that operates in the 1225–1525 MHz range, with the seven beams arranged in a hexagonal pattern. Each beam is approximately 3.5' wide. The receiver has a cold sky system temperature of ~30 K. The central beam has a gain of ~11 K/Jy , with the peripheral beams having a slightly lower gain. Some of the sidelobes of the ALFA beams are sensitive as well, with the peak of these sidelobes having a loss of only −8.5 dB, i.e., the gain at the sidelobe peak is only 1/7th of that at boresight of the central beam.' (Chennamangalam 2017)
    * Gain, Tsys, SEFD (http://www.naic.edu/alfa/performance/spectral-scans/wide/ , http://www.naic.edu/alfa/performance/)

* What is the efficiency of Arecibo?
    * wrapped into the SEFD measurements above

### Search Space

* ALFABURST search specs:
    * processes 56 MHz (1/5 of the band), 512 channel spectra every 256 microseconds, Stokes I only
    * astroaccelerate dedispersion: 32768 spectra (8.4 seconds) buffers

* We search a maximum DM of 10000, are we saving a large enough buffer if a 10000 DM FRB is detected? yes, just enough for the full band
    * kramer and lorimer eq 5.1 delta t ~ 4.15 * 10^6 ms * (f_ref^-2 - f_chan^-2) * DM, f_ref and f_chan in MHz
    * band: 300 MHz centered at 1375 MHz
    * DM_max = 10000
    * 56 MHz: 1375 - 28 = 1347, 1375 + 28 = 1403 ; delta t = 4.15 * 10^6 m * ( 1347^-2 - 1403^-2) * 10000 = 1789 ms = 1.789 s
    * 280 MHz: 1375 - 140 = 1235, 1375 + 140 = 1515 ; delta t = 4.15 * 10^6 m * ( 1235^-2 - 1515^-2) * 10000 = 9128 ms = 9.128 s

In [13]:
def timeBufferSize(f_start, f_stop, maxdm):
    """Kramer and Lorimer e.q. 5.1
    f_start, f_stop: start and stop frequency, in MHz
    maxdm: maximum search DM
    returns: delay (ms) between start and stop frequency given a maximum DM"""
    bw = f_stop - f_start
    deltat = 4.15 * (10.**6) * (f_start**(-2.) - f_stop**(-2.)) * maxdm
    return deltat

In [14]:
print 'L-band, 56 MHz (1/5th of band):', timeBufferSize(1347., 1403., 10000), 'ms'
print 'L-band, 280 MHz:', timeBufferSize(1235., 1515., 10000), 'ms'

L-band, 56 MHz (1/5th of band): 1789.4447095 ms
L-band, 280 MHz: 9128.08427428 ms


### Predicted Event Rates

* **OPEN** What is the basic detection rate give the dish size, system temp, observing time, beam pattern, assuming flat spectrum model? for a given spectral index?
    * see Lorimer et al 2013, ARTEMIS 2015 paper

From http://www.naic.edu/alfa/performance/spectral-scans/wide/:

* Central Beam (ID: 0):
    * Gain: 10-12 K/Jy
    * T_sys: 30 K
    * SEFD: 3 Jy
* Side Beam (ID: 1-6):
    * Gain: 9 K/Jy
    * T_sys: 30 K
    * SEFD: 4 Jy